In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def Init_plateau():
    plateau = [
        ['#', '#', '#', '#', '#', '#'],
        ['#', 'S', ' ', ' ', ' ', '#'],
        ['#', 'D', ' ', 'D', ' ', '#'],
        ['#', ' ', ' ', ' ', 'D', '#'],
        ['#', ' ', 'D', ' ', 'J', '#'],
        ['#', '#', '#', '#', '#', '#']
        ]
    return plateau;
_plateau = Init_plateau();
_player_position = [1,1];

_rewards = {
    " ":0,
    "D":-1,
    "J":1,
    "S":0,
    "#":0
    }

def list_contain(liste, item):
    for i in range(len(item)):
        if(liste[i] == item):
            return True
    return False

def get_reward(x, y):
    case = _plateau[y][x];
    return _rewards[case];

def check_currently_available_action():
    return check_available_action(_player_position[0], _player_position[1]);

def check_available_action(x, y):
    case_right = _plateau[y][x+1];
    case_down  = _plateau[y+1][x];
    case_left  = _plateau[y][x-1];
    case_up    = _plateau[y-1][x];
    
    actions = [];
    
    if(case_right != "#"): actions.append("right");
    if(case_down  != "#"): actions.append("down");
    if(case_left  != "#"): actions.append("left");
    if(case_up    != "#"): actions.append("up");
    
    return actions;
    
def play_action(action):
    if action == "right":
        _player_position[0] += 1;
    elif action == "left":
        _player_position[0] += 1;
    elif action == "down":
        _player_position[0] += 1;
    elif action == "up":
        _player_position[0] += 1;

           
Position_dragon = [2, 4, 11, 13];
Position_jail = 15;
Start_position = 0;


print(check_available_action(2,2));
print(get_reward(1, 1))
print(get_reward(1, 2))
print(get_reward(4, 4))
(play_action("right"));
(play_action("right"));
(play_action("down"));
print(check_currently_available_action());